<a href="https://colab.research.google.com/github/tekoryu/rough-on-rats/blob/main/notebooks/model_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Engenharia de Softwares Inteligentes

## TITANIC CHALLENGE

Desafio, dados originais e detalhes da base disponiveis em:

https://www.kaggle.com/competitions/titanic